# Diving into code approximations

Like any code, there are several approximations that go into computing intesity from various phase functions. In reflected light models, some of these approximations drastically change the output spectra. 

Here we go through each approximation to show what the effects are of each. Also, to show how to toggle between various inputs. 

In [1]:
import warnings
warnings.filterwarnings('ignore')
#picaso
from picaso import justdoit as jdi 
from picaso import justplotit as jpi 
from bokeh.plotting import show, figure
from bokeh.layouts import column
from bokeh.palettes import Colorblind8
from bokeh.io import output_notebook 
output_notebook()

Loading BokehJS ...

## Using the `approx` key

In [2]:
all_inputs = jdi.load_inputs()
print(all_inputs['approx'])

{'single_phase': 'TTHG', 'multi_phase': 'N=2', 'delta_eddington': True, 'raman': 'oklopcic'}


Notice that all the `approx` keys have predefined inputs. These are our recommendations for how to run the code. But, users should always be weary of these and test their sensitivity to your results.

## What approximations exist?

We can easily print out all our options for what approximations exist in the code

In [3]:
print('Options for Direct Scattring Phase: ', jdi.single_phase_options())

print('Options for Multiple Scattring Phase: ', jdi.multi_phase_options())

print('Options for Raman Scattring: ', jdi.raman_options())

Options for Direct Scattring Phase:  ['cahoy', 'OTHG', 'TTHG', 'TTGH_ray']
Options for Multiple Scattring Phase:  ['N=2', 'N=1']
Options for Raman Scattring:  ['oklopcic', 'pollack', 'none']


### Set inputs normally

In [4]:
#set planet 
all_inputs['planet']['gravity'] = 25
all_inputs['planet']['gravity_unit'] = 'm/(s**2)' 

#set star 
all_inputs['star']['temp'] = 6000
all_inputs['star']['metal'] = 0.0122
all_inputs['star']['logg'] = 4.437

#set atmosphere comp and temp 
all_inputs['atmosphere']['profile']['filepath'] = jdi.jupiter_pt()
#make a copy to have a separate cloud input dict
from copy import deepcopy
all_inputs_cld=deepcopy(all_inputs)
all_inputs_cld['atmosphere']['clouds']['filepath'] = jdi.jupiter_cld()

## Direct Scattering Approximation

The [derivation documentation](https://natashabatalha.github.io/picaso/picaso#slide02) has a full description of these direct scattering approximations. Briefly I'll describe them here: 

At the center of each is the [One Term Henyey-Greenstein Phase Function (OTHG)](http://adsabs.harvard.edu/abs/1941ApJ....93...70H) and the [Two Term HG Phase Function (TTHG)](http://adsabs.harvard.edu/abs/1965ApJ...142.1563I). 

We also know that planet atmospheres have high degrees of Rayleigh scattering. [Cahoy+2010](http://adsabs.harvard.edu/abs/2010ApJ...724..189C) developed a methodology for incorporating Rayleigh into the direct scattering component. 

A more rrobust way of dealing with Rayleigh is to directly fold it's phase function into the TTHG phase function (TTHG_Ray). 

We'll run each case with and without a cloud so you can see what happens in both regimes

In [5]:
#let's make two different figures for this
fig_cloudy = figure(x_range=[0.3,1],y_range=[0,1.2], width=500, height=300
                    ,x_axis_label='Wavelength [μm]', y_axis_label='Geometric Albedo',)
fig_no_cloud = figure(x_range=[0.3,1],y_range=[0,1.2], width=500, height=300
                      ,x_axis_label='Wavelength [μm]', y_axis_label='Geometric Albedo',)

#define our options
options = jdi.single_phase_options()
colors = Colorblind8[0:len(options)]

#loop through all approximations 
for approx, c in zip(options, colors):
    all_inputs['approx']['single_phase']     = approx
    all_inputs_cld['approx']['single_phase'] = approx
    wno_nc, alb_nc = jdi.picaso(all_inputs, 0)
    wno_c, alb_c = jdi.picaso(all_inputs_cld, 0)
    fig_no_cloud.line(1e4/wno_nc, alb_nc, legend=approx, color=c, line_width=3)
    fig_cloudy.line(1e4/wno_c, alb_c,  color=c, line_width=3)
jpi.plot_format(fig_cloudy)
jpi.plot_format(fig_no_cloud)
show(column(fig_no_cloud,fig_cloudy ))

## Multiple Scattering Approximations

Again, [derivation documentation](https://natashabatalha.github.io/picaso/picaso#slide03) has a full description of these multiple scattering approximations. 

To complete the multiple scattering integration over all _diffuse angles_, we have to use some mathematical tricks. [Legendre Polynomials](http://mathworld.wolfram.com/LegendrePolynomial.html) are often used to complete this integration to varying degrees. For Solar System/Exoplanet papers, we often stop the expansion at either `N=1` or `N=2`. Our standard will be to run with `N=2`, but below we show how to run each.

In [6]:
fig = figure(x_range=[0.3,1],y_range=[0,1.2], width=500, height=300,
             x_axis_label='Wavelength [μm]', y_axis_label='Geometric Albedo',)

options = jdi.multi_phase_options()
colors = Colorblind8[0:len(options)*2]

for approx, c1,c2 in zip(options, colors[0:2], colors[2:]):
    all_inputs['approx']['multi_phase']     = approx
    all_inputs_cld['approx']['multi_phase'] = approx
    wno_nc, alb_nc = jdi.picaso(all_inputs, 0)
    wno_c, alb_c = jdi.picaso(all_inputs_cld, 0)
    fig.line(1e4/wno_nc, alb_nc, color=c1, line_width=3)
    fig.line(1e4/wno_c, alb_c,  color=c2, line_width=3)
jpi.plot_format(fig)
show(fig)

## Raman Scattering Approximations

We all know the importance of Rayleigh scattering in planetary atmospheres. Raman scattering also has important implications for our spectra (these features have been observed in Solar System planets). In particular, at short wavelengths, Raman scattering imprints molecular features from the star on the planetary spectrum. 

The most complete analysis of all Raman approximations is in [Sromosvky+2005](http://adsabs.harvard.edu/abs/2005Icar..173..254S). From these, we use the _Pollack Approximation_ that was used in [Cahoy+2010](http://adsabs.harvard.edu/abs/2010ApJ...724..189C) and others. 

We include the original Pollack methodology, but also include a modified version with [Oklopcic et al 2018](http://iopscience.iop.org/article/10.3847/0004-637X/832/1/30/meta) cross sections and updated methodology to include effects of stellar spectrum.  

In [7]:
fig_cloudy = figure(x_range=[0.3,1],y_range=[0,1.2], width=500, height=300
                    ,x_axis_label='Wavelength [μm]', y_axis_label='Geometric Albedo',)
fig_no_cloud = figure(x_range=[0.3,1],y_range=[0,1.2], width=500, height=300
                      ,x_axis_label='Wavelength [μm]', y_axis_label='Geometric Albedo',)

options = jdi.raman_options()
colors = Colorblind8[0:len(options)]

for approx, c in zip(options, colors):
    all_inputs['approx']['raman']     = approx
    all_inputs_cld['approx']['raman'] = approx
    wno_nc, alb_nc = jdi.picaso(all_inputs, 0)
    wno_c, alb_c = jdi.picaso(all_inputs_cld, 0)
    fig_no_cloud.line(1e4/wno_nc, alb_nc, legend=approx, color=c, line_width=3)
    fig_cloudy.line(1e4/wno_c, alb_c,  color=c, line_width=3)

jpi.plot_format(fig_cloudy)
jpi.plot_format(fig_no_cloud)
show(column(fig_no_cloud,fig_cloudy ))

## The Effect of Stellar Spectrum

With the updated Raman scattering approximation, you will notice imprints of the stellar spectrum in the planet reflected light spectrum. Take a look below.

In [8]:
fig_cloudy = figure(x_range=[0.3,1],y_range=[0,1.2], width=500, height=300
                    ,x_axis_label='Wavelength [μm]', y_axis_label='Geometric Albedo',)
fig_no_cloud = figure(x_range=[0.3,1],y_range=[0,1.2], width=500, height=300
                      ,x_axis_label='Wavelength [μm]', y_axis_label='Geometric Albedo',)

options = [6000,4000,3500]
colors = Colorblind8[0:len(options)]

all_inputs['approx']['raman']     = 'oklopcic'
all_inputs_cld['approx']['raman'] = 'oklopcic'

for approx, c in zip(options, colors):
    all_inputs['star']['temp']     = approx
    all_inputs_cld['star']['temp']     = approx
    wno_nc, alb_nc = jdi.picaso(all_inputs, 0)
    wno_c, alb_c = jdi.picaso(all_inputs_cld, 0)
    fig_no_cloud.line(1e4/wno_nc, alb_nc, legend=str(approx), color=c, line_width=3)
    fig_cloudy.line(1e4/wno_c, alb_c,  color=c, line_width=3)

all_inputs['approx']['raman']     = 'pollack'
all_inputs_cld['approx']['raman']     = 'pollack'
wno_nc, alb_nc = jdi.picaso(all_inputs, 0)
wno_c, alb_c = jdi.picaso(all_inputs_cld, 0)
fig_no_cloud.line(1e4/wno_nc, alb_nc, legend='Pollack', color='black', line_width=2, line_dash='dashed')
fig_cloudy.line(1e4/wno_c, alb_c,  color='black', line_width=2, line_dash='dashed')


jpi.plot_format(fig_cloudy)
jpi.plot_format(fig_no_cloud)
show(column(fig_no_cloud,fig_cloudy ))